# Null Models

Here we will review a few null models and show you how to use software for fitting the power-law

First let's import the modules we always use.

In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

And then change the default settings for matplotlib as before

In [2]:
# change defaults to be less ugly
mpl.rc('xtick', labelsize=14, color="#222222") 
mpl.rc('ytick', labelsize=14, color="#222222") 
mpl.rc('font', **{'family':'sans-serif','sans-serif':['Arial']})
mpl.rc('font', size=16)
mpl.rc('xtick.major', size=6, width=1)
mpl.rc('xtick.minor', size=3, width=1)
mpl.rc('ytick.major', size=6, width=1)
mpl.rc('ytick.minor', size=3, width=1)
mpl.rc('axes', linewidth=1, edgecolor="#222222", labelcolor="#222222")
mpl.rc('text', usetex=False, color="#222222")

# Fitting a power-law using the Clauset et al. Method

Unfortunately, the package Clauset recommends, doesn't work for Python 3. However Ed Bullmore and others put together another package available here: https://github.com/jeffalstott/powerlaw


First let's install the package using $\textbf{pip}$

In [3]:
!pip install powerlaw 

In [4]:
import powerlaw

Now let's check if it works. We'll generate a scale-free network (using our code from last lecture) and then see if we can recover the exponent.

Note that the two functions below are from the second hands-on lecture, so we won't spend much time on them.

In [8]:
def powerlaw_degree_sequence(n, gamma, k_min):
    """ Implements the method for generating power-law distributed numbers
    from uniformly-distributed numbers described in Clauset et al., 2009,
    appendix D"""
    r = np.random.uniform(0, 1, size=n)
    deg = np.floor((k_min-0.5)*(1.0 - r)**(-1.0/(gamma-1)) + 0.5)
    deg = list(map(int, deg))
    return deg

In [12]:
powerlaw_degree_sequence(1000,2.5,2)

[3,
 6,
 6,
 3,
 2,
 2,
 2,
 4,
 2,
 14,
 4,
 2,
 3,
 5,
 2,
 3,
 2,
 3,
 6,
 2,
 2,
 8,
 2,
 4,
 2,
 5,
 2,
 2,
 2,
 3,
 3,
 2,
 2,
 4,
 7,
 10,
 2,
 3,
 2,
 2,
 2,
 2,
 5,
 2,
 3,
 5,
 2,
 2,
 8,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 11,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 6,
 2,
 3,
 2,
 4,
 3,
 2,
 2,
 2,
 4,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 2,
 11,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 5,
 2,
 2,
 9,
 2,
 2,
 2,
 2,
 6,
 3,
 3,
 2,
 2,
 3,
 2,
 2,
 5,
 2,
 4,
 11,
 6,
 3,
 3,
 4,
 8,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 4,
 4,
 11,
 2,
 4,
 5,
 2,
 2,
 2,
 2,
 3,
 3,
 2,
 2,
 5,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 7,
 2,
 4,
 2,
 2,
 3,
 8,
 3,
 2,
 2,
 5,
 2,
 2,
 4,
 4,
 2,
 3,
 3,
 4,
 4,
 16,
 3,
 3,
 2,
 4,
 3,
 21,
 2,
 2,
 4,
 2,
 5,
 7,
 2,
 4,
 4,
 3,
 3,
 4,
 9,
 2,
 2,
 3,
 2,
 2,
 13,
 2,
 2,
 2,
 7,
 2,
 4,
 2,
 2,
 2,
 2,
 3,
 2,
 4,
 2,
 3,
 5,
 71,
 3,
 5,
 3,
 3,
 3,
 2,
 2,
 3,
 3,
 2,
 4,
 2,
 5,
 2,
 5,
 2,
 2,
 33,
 2,
 4,
 3,
 2,
 2,
 2,
 2,
 4,
 7,
 2,
 

In [13]:
def sf_net(n,  gamma, k_min):
    deg = powerlaw_degree_sequence(n, gamma, k_min)
    # sum of all degrees must be even. Why is that?
    if sum(deg) % 2 == 1:
        deg[0] += 1
    G = nx.configuration_model(deg)
    H = nx.Graph(G)
    H.remove_edges_from(H.selfloop_edges())
    return H

Let's generate a network of $n=20,000$ nodes, with $\gamma=2.5$ and $k_{min}=2$.

In [14]:
G=sf_net(20000, 2.5,2)

Now let's see what our powerlaw code would estimate the values of $\gamma$ and $k_{min}$ to be.

First we will need to get the degree sequence

The line of code below just takes the degree sequence and obtains the values of the degrees. There is probably a nicer single networkx command to do this, but here we just take the degree dictionary and then just get the 'values' of the degrees

In [20]:
degrees=np.array(list(dict(G.degree()).values()))

In [21]:
results = powerlaw.Fit(degrees)
print(results.power_law.alpha)
print(results.power_law.xmin)

2.5231666515676863
11.0


Calculating best minimal value for power law fit
//anaconda3/lib/python3.7/site-packages/powerlaw.py:700: RuntimeWarning: invalid value encountered in true_divide
  (Theoretical_CDF * (1 - Theoretical_CDF))


Note that xmin above, is not the same as kmin. This is because they reflect different things. xmin is the point from which the data displays the power-law behavior, not the minimal value in the network. The value of $\gamma$ is pretty close, however note that this is a finite system, so we will have some error.

Now let's compare to see if our power-law is actually a better fit than a different distribution e.g., a lognormal distribution.

The line of code below will do this for us:

In [22]:
R, p = results.distribution_compare('power_law', 'lognormal_positive')


R: Loglikelihood ratio of the two distributions’ fit to the data. If greater than 0, the first distribution is preferred. If less than 0, the second distribution is preferred.

p: Significance of R

See: https://pythonhosted.org/powerlaw/#powerlaw.Fit.distribution_compare



In [23]:
print(R,p)

22.260343617177494 0.0002273957218205971


So our distribution is definitely a power-law and not a log normal. Yay! We showed this rigorously and can now be convinced with our answer

# Using the Configuration Model as a Null Model 

Let's use one of networkx's built in networks to understand how using a configuration model as a null model could help us get context on values

In [47]:
L=nx.generators.social.les_miserables_graph()
L=nx.Graph(L)
L.remove_edges_from(L.selfloop_edges())

This network is the coappearance of characters in Les miserables- see https://networkx.github.io/documentation/stable/reference/generated/networkx.generators.social.les_miserables_graph.html

Let's see what the clustering coefficient is like:

In [59]:
les_mis_clustering=nx.average_clustering(L)
print(les_mis_clustering)

0.5731367499320134


Hmmm... so what does that number mean? 57% of possible triangles appear in the network. Is that a lot or a little?

Let's compare to a degree shuffled version of the network (configuration model)...

In [60]:
'''First we get the degree sequence'''
degree_sequence = [d for n, d in L.degree()]     

In [62]:
'''Then we build a configuration model based on that degree sequence'''
G = nx.configuration_model(degree_sequence)

In [63]:
'''And here we remove multilinks (two edges that are the same) 
and self-loops (a node linking to itself) '''
H = nx.Graph(G)
H.remove_edges_from(H.selfloop_edges())

Now let's see what the clustering of this network is like

In [65]:
null_clustering=nx.average_clustering(H)
print(null_clustering)

0.16858843997222858


The clustering coefficient of the null model is only 15.5%, so we could say that the network has some nontrivial clustering that is beyond what is likely based on the degree sequence alone.

Of course, this isn't surprising as we'd expect the characters to interact with their neighbors neighbors more than random. In fact, many of the links may actually refer to cases where 3 or more characters all appear together, making it trivial that they form a triangle.